In [ ]:
#Script para enviar tabela xlsx para SQL da CIDADE

import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text

conexaoDB = ('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-HV85ADA\\PROJETO_MERCADO;DATABASE=MERCADO_OLTP;Trusted_Connection=yes;')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conexaoDB}')

caminho = r"C:\\Users\\marce\\OneDrive\Arquivos Antigos\\Área de Trabalho\\Projeto Supermecado - Arquivos\\BANCO DE DADOS\\DADOS\\CIDADE\\"
arquivo = r"CIDADES.csv"
caminhocompleto = caminho+arquivo

df_cidades = pd.read_csv(caminhocompleto,delimiter=";")

estados_brasileiros = ["Acre","Alagoas","Amazonas","Amapa","Bahia","Ceara","Distrito Federal","Espirito Santo",
"Goias","Maranhao","Minas Gerais","Mato Grosso do Sul","Mato Grosso","Para","Paraiba","Pernambuco","Piaui",
"Parana","Rio de Janeiro","Rio Grande do Norte","Rondonia","Roraima","Rio Grande do Sul","Santa Catarina",
"Sergipe","Sao Paulo","Tocantins"]

siglas_estados = {
    "Acre": "AC",
    "Alagoas": "AL",
    "Amapa": "AP",
    "Amazonas": "AM",
    "Bahia": "BA",
    "Ceara": "CE",
    "Distrito Federal": "DF",
    "Espirito Santo": "ES",
    "Goias": "GO",
    "Maranhao": "MA",
    "Mato Grosso": "MT",
    "Mato Grosso do Sul": "MS",
    "Minas Gerais": "MG",
    "Para": "PA",
    "Paraiba": "PB",
    "Parana": "PR",
    "Pernambuco": "PE",
    "Piaui": "PI",
    "Rio de Janeiro": "RJ",
    "Rio Grande do Norte": "RN",
    "Rio Grande do Sul": "RS",
    "Rondonia": "RO",
    "Roraima": "RR",
    "Santa Catarina": "SC",
    "Sao Paulo": "SP",
    "Sergipe": "SE",
    "Tocantins": "TO"
}

regioes = {
    "Norte":        ["Acre", "Amapa", "Amazonas", "Para", "Rondonia", "Roraima", "Tocantins"],
    "Nordeste":     ["Alagoas", "Bahia", "Ceara", "Maranhao", "Paraiba", "Pernambuco", "Piaui", "Rio Grande do Norte", "Sergipe"],
    "Centro-Oeste": ["Distrito Federal", "Goias", "Mato Grosso", "Mato Grosso do Sul"],
    "Sudeste":      ["Espirito Santo", "Minas Gerais", "Rio de Janeiro", "Sao Paulo"],
    "Sul":          ["Parana", "Rio Grande do Sul", "Santa Catarina"]
}

df_EST = []

for estado in estados_brasileiros:
    for regiao, estados_regiao in regioes.items():
        if estado in estados_regiao:
            df_EST.append({
                "Nome":   estado,
                "UF":     siglas_estados[estado],
                "Regiao": regiao,
                "Pais":   "Brasil"
            })
            break

df_EST = pd.DataFrame(df_EST)

estados = df_cidades["COD_UF"].unique().tolist()
estados.sort()

id_estado = [indice + 1 for indice in range(len(estados))]

df_estados = pd.DataFrame({
                    'ESTADO':    estados,
                    'ID_ESTADO': id_estado,

})

mapeamento = dict(zip(df_estados['ESTADO'], df_estados['ID_ESTADO']))

df_cidades['ID_ESTADO'] = df_cidades['COD_UF'].map(mapeamento)



df_CIDADE = pd.DataFrame({
                'NOME':         df_cidades['NOME_MUN'],
                'ID_ESTADO':    df_cidades['ID_ESTADO'],

})

df_ESTADO = pd.DataFrame({
                'ESTADO':       df_EST['Nome'],
                'UF':           df_estados['ESTADO'],
                'REGIAO':       df_EST['Regiao'],
                'PAIS':         df_EST['Pais']

})

display(df_CIDADE)
display(df_ESTADO)

tipo_dados_cidade = {

    'NOME':         VARCHAR(50),
    'ID_ESTADO':    INTEGER

}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE CIDADE"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

df_CIDADE.to_sql(name='CIDADE',con=engine,if_exists='append',index=False,dtype=tipo_dados_cidade)

tipo_dados_estado = {

    'ESTADO':  VARCHAR(25),
    'UF':      VARCHAR(2),
    'REGIAO':  VARCHAR(20),
    'PAIS':    VARCHAR(20),

}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE ESTADO"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

df_ESTADO.to_sql(name='ESTADO',con=engine,if_exists='append',index=False,dtype=tipo_dados_estado)

engine.dispose()

In [ ]:
#script para gerar dataframe com maiores cidades

#fazer consulta no banco de dados e buscar a cidade e o idcidade, trazer para um df e filtrar pela lista maiores cidades
import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text

conexaoDB = ('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-HV85ADA\\PROJETO_MERCADO;DATABASE=MERCADO_OLTP;Trusted_Connection=yes;')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conexaoDB}')

#47 maiores cidades
cidades_brasil = [ "Sao Paulo", "Rio de Janeiro", "Brasilia", "Salvador", "Fortaleza",
    "Belo Horizonte", "Manaus", "Curitiba", "Recife", "Goiania",
    "Porto Alegre", "Guarulhos", "Campinas", "Sao Luis",
    "Sao Goncalo", "Maceio", "Duque de Caxias", "Natal", "Teresina",
    "Sao Bernardo do Campo", "Joao Pessoa", "Osasco",
    "Jaboatao dos Guararapes", "Ribeirao Preto", "Uberlandia", "Contagem", "Sorocaba",
    "Aracaju", "Feira de Santana", "Cuiaba", "Joinville", "Aparecida de Goiania",
    "Londrina", "Juiz de Fora", "Ananindeua", "Belford Roxo", "Niteroi",
    "Sao Joao de Meriti", "Campos dos Goytacazes", "Serra", "Caxias do Sul", "Maua",
    "Macapa", "Piracicaba", "Canoas", "Bauru", "Sao Jose dos Campos"
]

df_cidades = pd.read_sql("SELECT IDCIDADE, NOME FROM CIDADE", engine) #le e copia os dados do banco para o df

df_maiores_cidades = df_cidades[df_cidades["NOME"].isin(cidades_brasil)].copy() #filtra os valores do df pelo parametro lista de cidades

lista_maiores_cidades= df_maiores_cidades['IDCIDADE'].to_list() #converte a coluna do df em lista

#Comando simplificado que faz a consulta com o filtro de cidade, iterando pelos valores de cidade da lista e retorna diretamente uma lista de id_cidades
df = pd.read_sql("SELECT IDCIDADE FROM CIDADE WHERE NOME IN ({})".format(','.join(f"'{cidade}'" for cidade in cidades_brasil)), engine)['IDCIDADE'].tolist()

display(df)

display(lista_maiores_cidades)